In [1]:
try:
    import google.colab # type: ignore
    IN_COLAB = True
except:
    IN_COLAB = False

import os, sys

if IN_COLAB:
    # Code to download the necessary files (e.g. solutions, test funcs)
    if not os.path.exists("chapter1_transformers"):
        !curl -o /content/main.zip https://codeload.github.com/callummcdougall/ARENA_2.0/zip/refs/heads/main
        !unzip /content/main.zip 'ARENA_2.0-main/chapter1_transformers/exercises/*'
        sys.path.append("/content/ARENA_2.0-main/chapter1_transformers/exercises")
        os.remove("/content/main.zip")
        os.rename("ARENA_2.0-main/chapter1_transformers", "chapter1_transformers")
        os.rmdir("ARENA_2.0-main")

         # Install packages
        %pip install einops
        %pip install jaxtyping
        %pip install transformer_lens
        %pip install git+https://github.com/callummcdougall/CircuitsVis.git#subdirectory=python
        %pip install s3fs
        %pip install omegaconf
        %pip install git+https://github.com/CindyXWu/devinterp-automata.git
        %pip install torch-ema

        !curl -o /content/main.zip https://codeload.github.com/CindyXWu/devinterp-automata/zip/refs/heads/main
        !unzip -o /content/main.zip -d /content/

        sys.path.append("/content/devinterp-automata/")
        os.remove("/content/main.zip")

        os.chdir("chapter1_transformers/exercises")
else:
    from IPython import get_ipython
    ipython = get_ipython()
    ipython.run_line_magic("load_ext", "autoreload")
    ipython.run_line_magic("autoreload", "2")

    CHAPTER = r"chapter1_transformers"
    CHAPTER_DIR = r"./" if CHAPTER in os.listdir() else os.getcwd().split(CHAPTER)[0]
    EXERCISES_DIR = CHAPTER_DIR + f"{CHAPTER}/exercises"
    sys.path.append(EXERCISES_DIR)

In [2]:
from dotenv import load_dotenv
import plotly.express as px
from typing import List, Union, Optional, Dict, Tuple
from jaxtyping import Int, Float

import torch
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F

from pathlib import Path
import numpy as np
import pandas as pd
import einops
import re
import functools
from tqdm import tqdm
from IPython.display import display
import webbrowser
import gdown
from transformer_lens.hook_points import HookPoint
from transformer_lens import utils, HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache
from transformer_lens.utils import to_numpy

import circuitsvis as cv
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# For Dashiell's groups code
from copy import deepcopy
from functools import reduce
from itertools import product
import math
import numpy as np
from operator import mul
import torch

torch.set_grad_enabled(False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MAIN = __name__ == "__main__"

import wandb
from pathlib import Path
import os
import yaml
import s3fs
from omegaconf import OmegaConf

from di_automata.config_setup import *
from di_automata.constructors import (
    construct_model,
    create_dataloader_hf,
)
from di_automata.tasks.data_utils import take_n
import plotly.io as pio

# AWS
load_dotenv()
AWS_KEY, AWS_SECRET = os.getenv("AWS_KEY"), os.getenv("AWS_SECRET")
s3 = s3fs.S3FileSystem(key=AWS_KEY, secret=AWS_SECRET)

/Users/cindy/anaconda3/envs/devinterp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/cindy/anaconda3/envs/devinterp/lib/python3.12/site-packages/pydantic/_internal/_fields.py:151: UserWarning: Field "model_type" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/cindy/anaconda3/envs/devinterp/lib/python3.12/site-packages/pydantic/_internal/_fields.py:151: UserWarning: Field "model_save_method" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [3]:
def imshow_attention(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(utils.to_numpy(tensor), labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

In [4]:
update_layout_set = {"xaxis_range", "yaxis_range", "hovermode", "xaxis_title", "yaxis_title", "colorbar", "colorscale", "coloraxis", "title_x", "bargap", "bargroupgap", "xaxis_tickformat", "yaxis_tickformat", "title_y", "legend_title_text", "xaxis_showgrid", "xaxis_gridwidth", "xaxis_gridcolor", "yaxis_showgrid", "yaxis_gridwidth", "yaxis_gridcolor", "showlegend", "xaxis_tickmode", "yaxis_tickmode", "margin", "xaxis_visible", "yaxis_visible", "bargap", "bargroupgap", "coloraxis_showscale", "xaxis_tickangle"}
import torch as t

def imshow(tensor: t.Tensor, renderer=None, **kwargs):
    kwargs_post = {k: v for k, v in kwargs.items() if k in update_layout_set}
    kwargs_pre = {k: v for k, v in kwargs.items() if k not in update_layout_set}
    facet_labels = kwargs_pre.pop("facet_labels", None)
    border = kwargs_pre.pop("border", False)
    return_fig = kwargs_pre.pop("return_fig", False)
    text = kwargs_pre.pop("text", None)
    xaxis_tickangle = kwargs_post.pop("xaxis_tickangle", None)
    static = kwargs_pre.pop("static", False)
    if "color_continuous_scale" not in kwargs_pre:
        kwargs_pre["color_continuous_scale"] = "RdBu"
    if "color_continuous_midpoint" not in kwargs_pre:
        kwargs_pre["color_continuous_midpoint"] = 0.0
    if "margin" in kwargs_post and isinstance(kwargs_post["margin"], int):
        kwargs_post["margin"] = dict.fromkeys(list("tblr"), kwargs_post["margin"])
    fig = px.imshow(to_numpy(tensor), **kwargs_pre).update_layout(**kwargs_post)
    if facet_labels:
        # Weird thing where facet col wrap means labels are in wrong order
        if "facet_col_wrap" in kwargs_pre:
            facet_labels = reorder_list_in_plotly_way(facet_labels, kwargs_pre["facet_col_wrap"])
        for i, label in enumerate(facet_labels):
            fig.layout.annotations[i]['text'] = label
    if border:
        fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
        fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
    if text:
        if tensor.ndim == 2:
            # if 2D, then we assume text is a list of lists of strings
            assert isinstance(text[0], list)
            assert isinstance(text[0][0], str)
            text = [text]
        else:
            # if 3D, then text is either repeated for each facet, or different
            assert isinstance(text[0], list)
            if isinstance(text[0][0], str):
                text = [text for _ in range(len(fig.data))]
        for i, _text in enumerate(text):
            fig.data[i].update(
                text=_text,
                texttemplate="%{text}",
                textfont={"size": 12}
            )
    # Very hacky way of fixing the fact that updating layout with new tickangle only applies to first facet by default
    if xaxis_tickangle is not None:
        n_facets = 1 if tensor.ndim == 2 else tensor.shape[0]
        for i in range(1, 1+n_facets):
            xaxis_name = "xaxis" if i == 1 else f"xaxis{i}"
            fig.layout[xaxis_name]["tickangle"] = xaxis_tickangle
    return fig if return_fig else fig.show(renderer=renderer, config={"staticPlot": static})

def reorder_list_in_plotly_way(L: list, col_wrap: int):
    '''
    Helper function, because Plotly orders figures in an annoying way when there's column wrap.
    '''
    L_new = []
    while len(L) > 0:
        L_new.extend(L[-col_wrap:])
        L = L[:-col_wrap]
    print(f"Reordered labels: {L_new}")
    return L_new

In [5]:
DI_ROOT = Path("/content/devinterp-automata-main/") if IN_COLAB else Path("../")
config_file_path = DI_ROOT / f"scripts/configs/slt_config.yaml"
slt_config = OmegaConf.load(config_file_path)

with open(DI_ROOT / f"scripts/configs/task_config/{slt_config.dataset_type}.yaml", 'r') as file:
    task_config = yaml.safe_load(file)

In [6]:
OmegaConf.set_struct(slt_config, False) # Allow new configuration values to be added
# Because we are in Colab and not VSCode, here is where you want to edit your config values
task_config["n"] = 4
slt_config["task_config"] = task_config

# Convert OmegaConf object to MainConfig Pydantic model for dynamic type validation - NECESSARY DO NOT SKIP
pydantic_config = PostRunSLTConfig(**slt_config)
# Convert back to OmegaConf object for compatibility with existing code
slt_config = OmegaConf.create(pydantic_config.model_dump())

print(task_config["dataset_type"])

dihedral


In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Run path and name for easy referral later
run_path = f"{slt_config.entity_name}/{slt_config.wandb_project_name}"
run_name = slt_config.run_name
print(run_name)

dihedral_TF_LENS_LR0.0001_its15000_layers3_seqlen25_nstates4_prob1None_nactionsNone


In [8]:
# Get run information
api = wandb.Api(timeout=3000)
run_list = api.runs(
    path=run_path,
    filters={
        "display_name": run_name,
        "state": "finished",
        },
    order="created_at", # Default descending order so backwards in time
)
assert run_list, f"Specified run {run_name} does not exist"
run_api = run_list[slt_config.run_idx]
try: history = run_api.history()
except: history = run_api.history
loss_history = history["Train Loss"]
accuracy_history = history["Train Acc"]
steps = history["_step"]
time = run_api.config["time"]

In [9]:
def get_config() -> MainConfig:
    """"
    Manually get config from run as artifact.
    WandB also logs automatically for each run, but it doesn't log enums correctly.
    """
    artifact = api.artifact(f"{run_path}/config:{run_name}_{time}")
    data_dir = artifact.download()
    config_path = Path(data_dir) / "config.yaml"
    return OmegaConf.load(config_path)

In [10]:
config = get_config()

# Set total number of unique samples seen (n). If this is not done it will break LLC estimator.
slt_config.rlct_config.sgld_kwargs.num_samples = slt_config.rlct_config.num_samples = config.rlct_config.sgld_kwargs.num_samples
slt_config.nano_gpt_config = config.nano_gpt_config

wandb:   1 of 1 files downloaded.  


In [11]:
def restore_state_single_cp(cp_idx: int) -> dict:
    """Restore model state from a single checkpoint.
    Used in _load_logits_states() and _calculate_rlct().

    Args:
        idx_cp: index of checkpoint.

    Returns:
        model state dictionary.
    """
    idx = cp_idx * config.rlct_config.ed_config.eval_frequency * slt_config.skip_cps
    print(f"Getting checkpoint {idx}")
    print(config.model_save_method)
    match config.model_save_method:
        case "wandb":
            artifact = api.artifact(f"{run_path}/states:idx{idx}_{run_name}_{time}")
            data_dir = artifact.download()
            state_path = Path(data_dir) / f"states_{idx}.torch"
            states = torch.load(state_path)
        case "aws":
            with s3.open(f'{config.aws_bucket}/{run_name}_{time}/states_{idx}.pth', mode='rb') as file:
                states = torch.load(file, map_location=device)
    return states["model"]

In [12]:
current_directory = Path().absolute()
logits_file_path = current_directory.parent / f"di_automata/logits_{run_name}_{time}"
print(logits_file_path)

/Users/cindy/ml-projects/devinterp-automata/di_automata/logits_dihedral_TF_LENS_LR0.0001_its15000_layers3_seqlen25_nstates4_prob1None_nactionsNone_02_26_04_17


In [13]:
ed_loader = create_dataloader_hf(config, deterministic=True) # Make sure deterministic to see same data

In [14]:
def display_layer_heads(att, batch_idx=0):
  display(cv.attention.attention_patterns(
      tokens=list_of_strings(inputs[batch_idx,...]),
      attention=att[batch_idx,...],
      attention_head_names=[f"L0H{i}" for i in range(4)],
  ))
  # 0 is toggle action
  # 1 is drive action
  print(inputs[batch_idx,...])
  print(labels[batch_idx,...])

def list_of_strings(tensor):
  return tensor.numpy().astype(str).tolist()


def display_layer_heads_batch(att: torch.Tensor, cache: ActivationCache, toks: list[str]):
  """TODO: refactor"""
  cv.attention.from_cache(
    cache = cache,
    tokens = toks,
    batch_idx = list(range(10)),
    attention_type = "info-weighted",
    radioitems = True,
    return_mode = "view",
    batch_labels = lambda batch_idx, str_tok_list: format_sequence(str_tok_list, dataset.str_tok_labels[batch_idx]),
    mode = "small",
  )

# Get checkpoints

In [15]:
# Form 1
cp_idx_0 = 500
state_0 = restore_state_single_cp(cp_idx_0)
model_0, _ = construct_model(config)
model_0.load_state_dict(state_0)

# Form 2
cp_idx_1 = 1400
state_1 = restore_state_single_cp(cp_idx_1)
model_1, _ = construct_model(config)
model_1.load_state_dict(state_1)

# Pre-form
cp_idx_2 = 30
state_2 = restore_state_single_cp(cp_idx_2)
model_2, _ = construct_model(config)
model_2.load_state_dict(state_2)

Getting checkpoint 4000
aws
Getting checkpoint 11200
aws
Getting checkpoint 240
aws


<All keys matched successfully>

In [16]:
# Pass data through
for data in take_n(ed_loader, 1):
    inputs = data["input_ids"]
    labels = data["label_ids"]
    break

logits_0, cache_0 = model_0.run_with_cache(inputs)
logits_1, cache_1 = model_1.run_with_cache(inputs)
logits_2, cache_2 = model_2.run_with_cache(inputs)

# Attention patterns
## First layer attention

In [17]:
IDX = 8

In [18]:
att_2_0 = cache_2["pattern", 0, "attn"]
display_layer_heads(att_2_0, batch_idx=IDX)

tensor([0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
        1])
tensor([4, 7, 3, 0, 4, 0, 1, 2, 6, 5, 1, 2, 3, 7, 6, 2, 6, 2, 6, 5, 1, 5, 4, 0,
        1])


In [19]:
att_0_0 = cache_0["pattern", 0, "attn"]
display_layer_heads(att_0_0, batch_idx=IDX)

tensor([0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
        1])
tensor([4, 7, 3, 0, 4, 0, 1, 2, 6, 5, 1, 2, 3, 7, 6, 2, 6, 2, 6, 5, 1, 5, 4, 0,
        1])


In [20]:
att_1_0 = cache_1["pattern", 0, "attn"]
# for head in range(config.tflens_config.n_heads):
#   imshow_attention(att_1[0,head,...])
display_layer_heads(att_1_0, batch_idx=IDX)

tensor([0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
        1])
tensor([4, 7, 3, 0, 4, 0, 1, 2, 6, 5, 1, 2, 3, 7, 6, 2, 6, 2, 6, 5, 1, 5, 4, 0,
        1])


## Second layer attention

In [21]:
att_0_2 = cache_2["pattern", 1, "attn"]
display_layer_heads(att_0_2, batch_idx=IDX)

tensor([0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
        1])
tensor([4, 7, 3, 0, 4, 0, 1, 2, 6, 5, 1, 2, 3, 7, 6, 2, 6, 2, 6, 5, 1, 5, 4, 0,
        1])


In [22]:
att_0_1 = cache_0["pattern", 1, "attn"]
display_layer_heads(att_0_1, batch_idx=IDX)

tensor([0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
        1])
tensor([4, 7, 3, 0, 4, 0, 1, 2, 6, 5, 1, 2, 3, 7, 6, 2, 6, 2, 6, 5, 1, 5, 4, 0,
        1])


In [23]:
att_1_1 = cache_1["pattern", 1, "attn"]
display_layer_heads(att_1_1, batch_idx=IDX)

tensor([0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
        1])
tensor([4, 7, 3, 0, 4, 0, 1, 2, 6, 5, 1, 2, 3, 7, 6, 2, 6, 2, 6, 5, 1, 5, 4, 0,
        1])


## Third layer attention

In [24]:
att_2_2 = cache_2["pattern", 2, "attn"]
display_layer_heads(att_2_2, batch_idx=IDX)
display_layer_heads(att_2_2, batch_idx=0)

tensor([0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
        1])
tensor([4, 7, 3, 0, 4, 0, 1, 2, 6, 5, 1, 2, 3, 7, 6, 2, 6, 2, 6, 5, 1, 5, 4, 0,
        1])


tensor([0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1,
        1])
tensor([4, 7, 6, 2, 6, 5, 1, 2, 6, 2, 3, 0, 1, 2, 3, 0, 1, 5, 4, 0, 1, 5, 1, 2,
        3])


In [25]:
att_0_2 = cache_0["pattern", 2, "attn"]
display_layer_heads(att_0_2, batch_idx=IDX)
display_layer_heads(att_0_2, batch_idx=0)

tensor([0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
        1])
tensor([4, 7, 3, 0, 4, 0, 1, 2, 6, 5, 1, 2, 3, 7, 6, 2, 6, 2, 6, 5, 1, 5, 4, 0,
        1])


tensor([0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1,
        1])
tensor([4, 7, 6, 2, 6, 5, 1, 2, 6, 2, 3, 0, 1, 2, 3, 0, 1, 5, 4, 0, 1, 5, 1, 2,
        3])


In [26]:
att_1_2 = cache_1["pattern", 2, "attn"]
display_layer_heads(att_1_2, batch_idx=IDX)
display_layer_heads(att_1_2, batch_idx=0)

tensor([0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
        1])
tensor([4, 7, 3, 0, 4, 0, 1, 2, 6, 5, 1, 2, 3, 7, 6, 2, 6, 2, 6, 5, 1, 5, 4, 0,
        1])


tensor([0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1,
        1])
tensor([4, 7, 6, 2, 6, 5, 1, 2, 6, 2, 3, 0, 1, 2, 3, 0, 1, 5, 4, 0, 1, 5, 1, 2,
        3])


In [27]:
inputs_dict = {i: inputs[0][i].item() for i in range(inputs[0].shape[0])}
labels_dict = {i: labels[0][i].item() for i in range(labels[0].shape[0])}

for head in range(config.tflens_config.n_heads):
  imshow_attention(att_1_2[0,head,...])
  print(inputs_dict)
  print(labels_dict)

{0: 0, 1: 1, 2: 1, 3: 0, 4: 0, 5: 1, 6: 0, 7: 1, 8: 0, 9: 0, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 0, 18: 1, 19: 0, 20: 1, 21: 0, 22: 0, 23: 1, 24: 1}
{0: 4, 1: 7, 2: 6, 3: 2, 4: 6, 5: 5, 6: 1, 7: 2, 8: 6, 9: 2, 10: 3, 11: 0, 12: 1, 13: 2, 14: 3, 15: 0, 16: 1, 17: 5, 18: 4, 19: 0, 20: 1, 21: 5, 22: 1, 23: 2, 24: 3}


{0: 0, 1: 1, 2: 1, 3: 0, 4: 0, 5: 1, 6: 0, 7: 1, 8: 0, 9: 0, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 0, 18: 1, 19: 0, 20: 1, 21: 0, 22: 0, 23: 1, 24: 1}
{0: 4, 1: 7, 2: 6, 3: 2, 4: 6, 5: 5, 6: 1, 7: 2, 8: 6, 9: 2, 10: 3, 11: 0, 12: 1, 13: 2, 14: 3, 15: 0, 16: 1, 17: 5, 18: 4, 19: 0, 20: 1, 21: 5, 22: 1, 23: 2, 24: 3}


{0: 0, 1: 1, 2: 1, 3: 0, 4: 0, 5: 1, 6: 0, 7: 1, 8: 0, 9: 0, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 0, 18: 1, 19: 0, 20: 1, 21: 0, 22: 0, 23: 1, 24: 1}
{0: 4, 1: 7, 2: 6, 3: 2, 4: 6, 5: 5, 6: 1, 7: 2, 8: 6, 9: 2, 10: 3, 11: 0, 12: 1, 13: 2, 14: 3, 15: 0, 16: 1, 17: 5, 18: 4, 19: 0, 20: 1, 21: 5, 22: 1, 23: 2, 24: 3}


{0: 0, 1: 1, 2: 1, 3: 0, 4: 0, 5: 1, 6: 0, 7: 1, 8: 0, 9: 0, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 0, 18: 1, 19: 0, 20: 1, 21: 0, 22: 0, 23: 1, 24: 1}
{0: 4, 1: 7, 2: 6, 3: 2, 4: 6, 5: 5, 6: 1, 7: 2, 8: 6, 9: 2, 10: 3, 11: 0, 12: 1, 13: 2, 14: 3, 15: 0, 16: 1, 17: 5, 18: 4, 19: 0, 20: 1, 21: 5, 22: 1, 23: 2, 24: 3}


In [28]:
for head in range(config.tflens_config.n_heads):
  imshow_attention(att_1_1[0,head,...])
  print(inputs_dict)
  print(labels_dict)

{0: 0, 1: 1, 2: 1, 3: 0, 4: 0, 5: 1, 6: 0, 7: 1, 8: 0, 9: 0, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 0, 18: 1, 19: 0, 20: 1, 21: 0, 22: 0, 23: 1, 24: 1}
{0: 4, 1: 7, 2: 6, 3: 2, 4: 6, 5: 5, 6: 1, 7: 2, 8: 6, 9: 2, 10: 3, 11: 0, 12: 1, 13: 2, 14: 3, 15: 0, 16: 1, 17: 5, 18: 4, 19: 0, 20: 1, 21: 5, 22: 1, 23: 2, 24: 3}


{0: 0, 1: 1, 2: 1, 3: 0, 4: 0, 5: 1, 6: 0, 7: 1, 8: 0, 9: 0, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 0, 18: 1, 19: 0, 20: 1, 21: 0, 22: 0, 23: 1, 24: 1}
{0: 4, 1: 7, 2: 6, 3: 2, 4: 6, 5: 5, 6: 1, 7: 2, 8: 6, 9: 2, 10: 3, 11: 0, 12: 1, 13: 2, 14: 3, 15: 0, 16: 1, 17: 5, 18: 4, 19: 0, 20: 1, 21: 5, 22: 1, 23: 2, 24: 3}


{0: 0, 1: 1, 2: 1, 3: 0, 4: 0, 5: 1, 6: 0, 7: 1, 8: 0, 9: 0, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 0, 18: 1, 19: 0, 20: 1, 21: 0, 22: 0, 23: 1, 24: 1}
{0: 4, 1: 7, 2: 6, 3: 2, 4: 6, 5: 5, 6: 1, 7: 2, 8: 6, 9: 2, 10: 3, 11: 0, 12: 1, 13: 2, 14: 3, 15: 0, 16: 1, 17: 5, 18: 4, 19: 0, 20: 1, 21: 5, 22: 1, 23: 2, 24: 3}


{0: 0, 1: 1, 2: 1, 3: 0, 4: 0, 5: 1, 6: 0, 7: 1, 8: 0, 9: 0, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 0, 18: 1, 19: 0, 20: 1, 21: 0, 22: 0, 23: 1, 24: 1}
{0: 4, 1: 7, 2: 6, 3: 2, 4: 6, 5: 5, 6: 1, 7: 2, 8: 6, 9: 2, 10: 3, 11: 0, 12: 1, 13: 2, 14: 3, 15: 0, 16: 1, 17: 5, 18: 4, 19: 0, 20: 1, 21: 5, 22: 1, 23: 2, 24: 3}


# OV circuits
## Post form 2

In [29]:
print(cache_1["scale"].shape)
# Layernorm scale, [batch, pos, 1]

torch.Size([64, 25, 1])


In [30]:
# [nlayers nheads dmodel dhead] x [nlayers nheads dmodel dhead].T
W_OV = model_0.W_V @ model_0.W_O # [nlayers nheads dmodel dmodel]
W_E = model_0.W_E # [vocab_in dhead]
W_U = model_0.W_U # [vocab_out dhead]
print(W_E.shape)

torch.Size([8, 256])


In [31]:
scale_final = cache_1["scale"][:, :, 0].mean()
scale_0 = cache_1["scale", 0, "ln1"].mean()
scale_1 = cache_1["scale", 1, "ln1"].mean()
scale_2 = cache_1["scale", 2, "ln1"].mean()

print(cache_1["scale", 2, "ln1"].shape)

torch.Size([64, 25, 1])


In [32]:
print(W_OV[2].shape)
print(W_OV[1].shape)
print(W_OV[0].shape)
print(W_OV.shape)


torch.Size([4, 256, 256])
torch.Size([4, 256, 256])
torch.Size([4, 256, 256])
torch.Size([3, 4, 256, 256])


In [33]:
# ! Get direct path
W_E_OV_direct = (W_E / scale_final) @ W_U
print(f"Direct {W_E_OV_direct.shape}") # [vocab_out vocab_out]

# ! Get full OV matrix for path through just layer 0
W_E_OV_0 = (W_E / scale_0) @ W_OV[0]
W_OV_0_full = (W_E_OV_0 / scale_final) @ W_U # [n_head vocab_in vocab_out]
print(f"Layer 0 {W_OV_0_full.shape}")

# ! Get full OV matrix for path through just layer 1
W_E_OV_1 = (W_E / scale_1) @ W_OV[1]
W_OV_1_full = (W_E_OV_1 / scale_final) @ W_U # [n_head vocab_in vocab_out]
print(f"Layer 1 {W_OV_1_full.shape}")

# ! Get full OV matrix for path through just layer 2
W_E_OV_2 = (W_E / scale_2) @ W_OV[2]
W_OV_2_full = (W_E_OV_2 / scale_final) @ W_U # [n_head vocab_in vocab_out]
print(f"Layer 2 {W_OV_2_full.shape}")

# ! Get full OV matrix for path through heads in layer 0 and 1
W_E_OV_01 = einops.einsum(
    (W_E_OV_0 / scale_1), W_OV[1],
    "head0 vocab_in d_model_in, head1 d_model_in d_model_out -> head0 head1 vocab_in d_model_out",
)
W_OV_01_full = (W_E_OV_01 / scale_final) @ W_U # [head0 head1 vocab_in vocab_out]
print(f"Layers 0, 1 {W_OV_01_full.shape}")

# ! Get full OV matrix for path through heads in layer 0 and 1
W_E_OV_02 = einops.einsum(
    (W_E_OV_0 / scale_1), W_OV[2],
    "head0 vocab_in d_model_in, head1 d_model_in d_model_out -> head0 head1 vocab_in d_model_out",
)
W_OV_02_full = (W_E_OV_02 / scale_final) @ W_U # [head0 head1 vocab_in vocab_out]

# Get full OV matrix for paths through heads in layer 1 and 2
W_E_OV_12 = einops.einsum(
    (W_E_OV_1 / scale_1), W_OV[2],
    "head0 vocab_in d_model_in, head1 d_model_in d_model_out -> head0 head1 vocab_in d_model_out",
)
W_OV_12_full = (W_E_OV_12 / scale_final) @ W_U # [head0 head1 vocab_in vocab_out]

# ! Get full OV matrix for path through heads in layer 0, 1, and 2
W_E_OV_012 = einops.einsum(
    (W_E_OV_01 / scale_2), W_OV[2],
    "head0 head1 vocab_in d_model_in, head2 d_model_in d_model_out -> head0 head1 head2 vocab_in d_model_out",
)
W_OV_012_full = (W_E_OV_012 / scale_final) @ W_U # [head0 head1 head2 vocab_in vocab_out]
print(f"All layers {W_OV_012_full.shape}")

Direct torch.Size([8, 8])
Layer 0 torch.Size([4, 8, 8])
Layer 1 torch.Size([4, 8, 8])
Layer 2 torch.Size([4, 8, 8])
Layers 0, 1 torch.Size([4, 4, 8, 8])
All layers torch.Size([4, 4, 4, 8, 8])


In [34]:
print(W_E_OV_direct[None, None].shape)
print(W_OV_0_full[:, None].shape)
print(W_OV_1_full[None].shape)
print(W_OV_2_full[None].shape)
print(W_OV_01_full.shape)

cat_1 = torch.cat([W_E_OV_direct[None, None], W_OV_0_full[:, None]]) # [head0 1 vocab_in vocab_out]
cat_2 = torch.cat([W_OV_1_full[None], W_OV_01_full])  # [head0 head1 vocab_in vocab_out]
print(cat_1.shape, cat_2.shape)

cat_3 = torch.cat([W_OV_02_full, W_OV_12_full])  # [head1 head2 vocab_in vocab_out]
print(cat_3.shape)
# cat_4 = torch.cat([W_OV_full[None], W_OV_2_full[None]]) # [head1 head2 vocab_in_ vocab_out]
# print(cat_3.shape, cat_4.shape)

W_OV_full_all = torch.cat([
    cat_1,
    cat_2,
], dim=1) # [head0 head1 vocab_in vocab_out]
# assert W_OV_full_all.shape == (5, 5, 8, 8)

torch.Size([1, 1, 8, 8])
torch.Size([4, 1, 8, 8])
torch.Size([1, 4, 8, 8])
torch.Size([1, 4, 8, 8])
torch.Size([4, 4, 8, 8])
torch.Size([5, 1, 8, 8]) torch.Size([5, 4, 8, 8])
torch.Size([8, 4, 8, 8])


In [35]:
tokens = [str(i) for i in range(8)]
components_0 = ["W<sub>E</sub>"] + [f"0.{i}" for i in range(4)]
components_1 = ["W<sub>U</sub>"] + [f"1.{i}" for i in range(4)]
components_2 = ["W<sub>U</sub>"] + [f"2.{i}" for i in range(4)]

# Using dict.fromkeys() prevents repeats
facet_labels = [" ➔ ".join(list(dict.fromkeys(["W<sub>E</sub>", c0, c1, "W<sub>U</sub>"]))) for c1 in components_1 for c0 in components_0]
imshow(
    W_OV_full_all.transpose(0, 1).flatten(0, 1), # .softmax(dim=-1),
    facet_col = 0,
    facet_col_wrap = 5,
    facet_labels = facet_labels,
    title = f"Full virtual OV circuits",
    x = tokens,
    y = tokens,
    labels = {"x": "Source", "y": "Dest"},
    height = 1200,
    width = 1200,
    # text = text,
)

Reordered labels: ['W<sub>E</sub> ➔ 1.3 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 0.0 ➔ 1.3 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 0.1 ➔ 1.3 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 0.2 ➔ 1.3 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 0.3 ➔ 1.3 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 1.2 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 0.0 ➔ 1.2 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 0.1 ➔ 1.2 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 0.2 ➔ 1.2 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 0.3 ➔ 1.2 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 1.1 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 0.0 ➔ 1.1 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 0.1 ➔ 1.1 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 0.2 ➔ 1.1 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 0.3 ➔ 1.1 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 1.0 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 0.0 ➔ 1.0 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 0.1 ➔ 1.0 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 0.2 ➔ 1.0 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 0.3 ➔ 1.0 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 0.0 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 0.1 ➔ W<sub>U</s

In [36]:
cat_3 = torch.cat([W_E_OV_direct[None, None], W_OV_0_full[:, None]])  # [head1 head2 vocab_in vocab_out]
cat_4 = torch.cat([W_OV_2_full[None], W_OV_02_full]) # [head1 head2 vocab_in_ vocab_out]

W_OV_full_all = torch.cat([
    cat_3,
    cat_4,
], dim=1) # [head0 head1 vocab_in vocab_out]

# Using dict.fromkeys() prevents repeats
facet_labels = [" ➔ ".join(list(dict.fromkeys(["W<sub>E</sub>", c0, c2, "W<sub>U</sub>"]))) for c2 in components_2 for c0 in components_0]
imshow(
    W_OV_full_all.transpose(0, 1).flatten(0, 1), # .softmax(dim=-1),
    facet_col = 0,
    facet_col_wrap = 5,
    facet_labels = facet_labels,
    title = f"Full virtual OV circuits",
    x = tokens,
    y = tokens,
    labels = {"x": "Source", "y": "Dest"},
    height = 1200,
    width = 1200,
    # text = text,
)

Reordered labels: ['W<sub>E</sub> ➔ 2.3 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 0.0 ➔ 2.3 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 0.1 ➔ 2.3 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 0.2 ➔ 2.3 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 0.3 ➔ 2.3 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 2.2 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 0.0 ➔ 2.2 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 0.1 ➔ 2.2 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 0.2 ➔ 2.2 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 0.3 ➔ 2.2 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 2.1 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 0.0 ➔ 2.1 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 0.1 ➔ 2.1 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 0.2 ➔ 2.1 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 0.3 ➔ 2.1 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 2.0 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 0.0 ➔ 2.0 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 0.1 ➔ 2.0 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 0.2 ➔ 2.0 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 0.3 ➔ 2.0 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 0.0 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 0.1 ➔ W<sub>U</s

In [37]:
components_1 = ["W<sub>E</sub>"] + [f"1.{i}" for i in range(4)]
components_2 = ["W<sub>U</sub>"] + [f"2.{i}" for i in range(4)]

cat_3 = torch.cat([W_E_OV_direct[None, None], W_OV_1_full[:, None]])  # [head1 head2 vocab_in vocab_out]
cat_4 = torch.cat([W_OV_2_full[None], W_OV_12_full]) # [head1 head2 vocab_in_ vocab_out]

W_OV_full_all = torch.cat([
    cat_3,
    cat_4,
], dim=1) # [head0 head1 vocab_in vocab_out]

# Using dict.fromkeys() prevents repeats
facet_labels = [" ➔ ".join(list(dict.fromkeys(["W<sub>E</sub>", c1, c2, "W<sub>U</sub>"]))) for c2 in components_2 for c1 in components_1]
imshow(
    W_OV_full_all.transpose(0, 1).flatten(0, 1), # .softmax(dim=-1),
    facet_col = 0,
    facet_col_wrap = 5,
    facet_labels = facet_labels,
    title = f"Full virtual OV circuits",
    x = tokens,
    y = tokens,
    labels = {"x": "Source", "y": "Dest"},
    height = 1200,
    width = 1200,
    # text = text,
)

Reordered labels: ['W<sub>E</sub> ➔ 2.3 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 1.0 ➔ 2.3 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 1.1 ➔ 2.3 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 1.2 ➔ 2.3 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 1.3 ➔ 2.3 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 2.2 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 1.0 ➔ 2.2 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 1.1 ➔ 2.2 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 1.2 ➔ 2.2 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 1.3 ➔ 2.2 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 2.1 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 1.0 ➔ 2.1 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 1.1 ➔ 2.1 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 1.2 ➔ 2.1 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 1.3 ➔ 2.1 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 2.0 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 1.0 ➔ 2.0 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 1.1 ➔ 2.0 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 1.2 ➔ 2.0 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 1.3 ➔ 2.0 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 1.0 ➔ W<sub>U</sub>', 'W<sub>E</sub> ➔ 1.1 ➔ W<sub>U</s

# Positional embeddings

In [38]:
w_pos_0 = model_0.W_pos
print(w_pos_0.shape)
imshow_attention(w_pos_0)

torch.Size([25, 256])


In [39]:
w_pos_1 = model_1.W_pos
imshow_attention(w_pos_1 - w_pos_0)

In [40]:
model_0.QK[0,...][0] @ w_pos_0.T

FactoredMatrix: Shape(torch.Size([256, 25])), Hidden Dim(64)